# Read out data from files

In [11]:
import json
import os
from collections import defaultdict

folder_path = os.path.join(os.getcwd(), '..', 'data', 'gdpr_data')

play_list = []

for filename in os.listdir(folder_path):

    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
         
        with open(file_path, 'r') as file:
            data = json.load(file)

            for entry in data:
                try:
                    track_id = entry['spotify_track_uri']
                    name = entry['master_metadata_track_name']
                    artist = entry['master_metadata_album_artist_name']
                    album = entry['master_metadata_album_album_name']
                    conn_country = entry['conn_country']
                    played_on = entry['ts']
                    played_track = {'track_id': track_id,
                                    'timestamp': played_on,
                                    'name': name,
                                    'album': album,
                                    'artist': artist,
                                    'played_from': conn_country
                                    }
                    play_list.append(played_track)
                    
                except Exception as e:
                    print(f'Missing field: {e}')
             


In [ ]:

# Sort the playlist by timestamp
play_list.sort(key=lambda x: x['timestamp'])

# Create a dictionary to store the number of times a track has been played
track_count = defaultdict(int)

for track in play_list:
    track_count[track['name'], track['artist']] += 1

# Make Track count readable in Data wrangler
track_count = [{'track': k, 'count': v} for k, v in track_count.items()]
